In [4]:
from trustyai.utils import TestModels
from trustyai.model import feature, output, Model, Dataset
from trustyai.model import simple_prediction
from trustyai.explainers import SHAPExplainer

import pandas as pd
import random

import keras

2024-12-08 19:11:08.558637: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-08 19:11:08.626383: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-08 19:11:08.626470: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-08 19:11:08.628154: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-08 19:11:08.638152: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-08 19:11:08.638900: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
X_train = pd.read_parquet("../2-dev_datascience/models/jukebox/1/artifacts/X_train.parquet")
X_test = pd.read_parquet("../2-dev_datascience/models/jukebox/1/artifacts/X_test.parquet")

In [6]:
model = keras.saving.load_model("../2-dev_datascience/models/jukebox/1/model.keras")

In [10]:
X_train

,is_explicit,duration_ms,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
644220,False,190560,0.841,0.783,6,-4.915,0,0.0432,0.1060,0.000002,0.0528,0.514,94.989
473163,False,146060,0.735,0.628,8,-5.572,0,0.2120,0.2580,0.000004,0.1000,0.192,103.925
585395,False,152078,0.760,0.747,0,-5.083,0,0.0282,0.0675,0.000003,0.3090,0.884,128.912
157601,True,170480,0.920,0.696,6,-3.356,0,0.0742,0.1830,0.000000,0.1490,0.545,106.966
839617,False,120600,0.858,0.677,8,-4.294,1,0.1180,0.1060,0.000000,0.2460,0.584,100.046
...,...,...,...,...,...,...,...,...,...,...,...,...,...
568129,True,197920,0.708,0.737,1,-4.045,1,0.0436,0.0739,0.001620,0.0955,0.607,91.986
285391,False,180530,0.492,0.686,5,-6.970,0,0.1400,0.2800,0.000000,0.1370,0.392,109.802
815084,False,190560,0.841,0.783,6,-4.915,0,0.0432,0.1060,0.000002,0.0528,0.514,94.989
722116,False,220750,0.593,0.354,4,-8.270,1,0.0279,0.6980,0.000036,0.2930,0.241,120.010


In [15]:
point_to_explain = X_train.iloc[0]
point_to_explain

is_explicit            False
duration_ms           190560
danceability           0.841
energy                 0.783
key                        6
loudness              -4.915
mode                       0
speechiness           0.0432
acousticness           0.106
instrumentalness    0.000002
liveness              0.0528
valence                0.514
tempo                 94.989
Name: 644220, dtype: object

In [ ]:
def pred(x):
    pred = model.predict(x)
    print(x)
    pred = np.squeeze(pred)
    pred = {output_names[i]: pred[i] for i in range(pred.shape[0])}
    print(pred["output-2"])
    return pd.DataFrame([pred])

In [17]:
from trustyai.model import Model
trustyai_model = Model(model.predict, dataframe_input=True, output_names=['output-1'])

In [23]:
trustyai_model(X_test.iloc[0:1])

ERROR:root: Fatal runtime error within the `predict_fun` supplied to trustyai.Model
ERROR:root: The error message has been captured and reproduced below:
ERROR:root: Traceback (most recent call last):
  File "/opt/app-root/lib64/python3.9/site-packages/trustyai/model/__init__.py", line 311, in wrapper
    return predict_fun(x)
  File "/opt/app-root/lib64/python3.9/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "/opt/app-root/lib64/python3.9/site-packages/tensorflow/python/framework/constant_op.py", line 103, in convert_to_eager_tensor
    return ops.EagerTensor(value, ctx.device_name, dtype)
ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type bool).



ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type bool).

In [18]:
from trustyai.explainers import SHAPExplainer
explainer = SHAPExplainer(background=X_train[:100])

In [22]:
explanations = explainer.explain(inputs=X_test.iloc[0:1], 
                                 outputs=1,
                                 model=trustyai_model)

java.util.concurrent.ExecutionException: java.util.concurrent.ExecutionException: java.lang.ClassCastException: class java.lang.Long cannot be cast to class java.lang.Double (java.lang.Long and java.lang.Double are in module java.base of loader 'bootstrap')